In [22]:
import os
import shutil
import glob
from mmcv import Config
import mmcv_custom   # noqa: F401,F403
import mmseg_custom   # noqa: F401,F403

from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel


import pandas as pd
import numpy as np
import json
import albumentations as A

In [26]:
# config file 들고오기
cfg = Config.fromfile('/opt/ml/ViT-Adapter/segmentation/configs/_sy_/10_uper_adapter_beit_L_focal_all/10_uper_adapter_beit_L_focal_all.py') # load config file
root='/opt/ml/input/data/mmseg/test'
# epoch = 'latest'

# multi_scale = [(x, x) for x in range(320,960+1,320)]
# print(multi_scale)

# dataset config 수정
cfg.work_dir = '/opt/ml/ViT-Adapter/segmentation/work_dirs/10_uper_adapter_beit_L_focal_all' # set work_dir
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (2560, 640) # Resize
# cfg.data.test.pipeline[1]['img_scale'] = multi_scale # Resize
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 1
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, 'best_mIoU_epoch_19.pth')


In [27]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2023-01-05 06:18:22,214 - mmseg - INFO - Loaded 819 images


In [28]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Position interpolate for blocks.0.attn.relative_position_bias_table from 27x27 to 79x79
x = [-39.00008316212981, -32.52029540928271, -26.974923021202137, -22.229218980359924, -18.16786741239758, -14.692181525406841, -11.717705619326258, -9.17216289667415, -6.993699208465826, -5.129380060092689, -3.533904355905343, -2.1685036277771, -1, 0, 1, 2.1685036277771, 3.533904355905343, 5.129380060092689, 6.993699208465826, 9.17216289667415, 11.717705619326258, 14.692181525406841, 18.16786741239758, 22.229218980359924, 26.974923021202137, 32.52029540928271, 39.00008316212981]
dx = [-39. -38. -37. -36. -35. -34. -33. -32. -31. -30. -29. -28. -27. -26.
 -25. -24. -23. -22. -21. -20. -19. -18. -17. -16. -15. -14. -13. -12.
 -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1.   0.   1.   2.
   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.
  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.
  31.  32.  33.  34.  35.  36.  37.  38.  39.]
Position interpolate

2023-01-05 06:18:31,375 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc_norm.weight, fc_norm.bias, head.weight, head.bias

missing keys in source state_dict: blocks.0.attn.relative_position_index, blocks.1.attn.relative_position_index, blocks.2.attn.relative_position_index, blocks.3.attn.relative_position_index, blocks.4.attn.relative_position_index, blocks.5.attn.relative_position_index, blocks.6.attn.relative_position_index, blocks.7.attn.relative_position_index, blocks.8.attn.relative_position_index, blocks.9.attn.relative_position_index, blocks.10.attn.relative_position_index, blocks.11.attn.relative_position_index, blocks.12.attn.relative_position_index, blocks.13.attn.relative_position_index, blocks.14.attn.relative_position_index, blocks.15.attn.relative_position_index, blocks.16.attn.relative_position_index, blocks.17.attn.relative_position_index, blocks.18.attn.relative_position_index, blocks.19.attn.relative_po

load checkpoint from local path: /opt/ml/ViT-Adapter/segmentation/work_dirs/10_uper_adapter_beit_L_focal_all/best_mIoU_epoch_19.pth


In [29]:
output = single_gpu_test(model, data_loader, out_dir='/opt/ml/out_dir')

[                                                  ] 0/819, elapsed: 0s, ETA:

/opt/conda/envs/openmmlab/lib/python3.8/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 2.1 task/s, elapsed: 395s, ETA:     0s

In [31]:
submission = pd.read_csv('/opt/ml/input/code/submission/sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# set resize
# input_size = 512
output_size = 256
transformed = A.Compose([A.Resize(output_size, output_size)])
		

# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    temp_mask = []
    mask = np.array(predict, dtype='uint8')
    mask = transformed(image=mask)
    temp_mask.append(mask['image'])
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join(cfg.work_dir, 'vit_focal_19_all_size.csv'), index=False)

print('Done')

/tmp/ipykernel_32575/414357301.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())},
/tmp/ipykernel_32575/414357301.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())},
/tmp/ipykernel_32575/414357301.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())},
/tmp/ipykernel_32575/414357301.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pa

Done
